In [46]:
import pandas as pd
from plotly import graph_objects as go
from plotly.subplots import make_subplots
import math
from ete3 import Tree
from Bio import AlignIO
from Bio.AlignIO.PhylipIO import RelaxedPhylipWriter
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment

In [28]:
def rf_distance_ete(t1, t2):
    rf, max_rf, common_leaves, parts_t1, parts_t2,discard_t1, discart_t2 = t1.robinson_foulds(t2, unrooted_trees = True)
    if max_rf == 0:
        print("?!")
        return 0
    return rf/max_rf


In [74]:
duplicates = pd.read_csv("duplicates.csv")
lang_data = pd.read_parquet("training_data/lang/binary.parquet")

In [81]:
for i,row in duplicates.iterrows():
    phlorest_name = row["phlorest_name"]
    lexibank_name = row["lexibank_name"]
    print(phlorest_name)
    print(lexibank_name)
    phlorest_align = AlignIO.read("alignments/lang/bin/" + phlorest_name, "phylip-relaxed")
    lexibank_align = AlignIO.read("alignments/lang/bin/" + lexibank_name, "phylip-relaxed")
    phlorest_taxa = set([phlorest_align[i].id for i in range(len(phlorest_align))])
    lexibank_taxa = set([lexibank_align[i].id for i in range(len(lexibank_align))])
    common_taxa = phlorest_taxa.intersection(lexibank_taxa)
    phlorest_taxa_only = phlorest_taxa.difference(common_taxa)
    lexibank_taxa_only = lexibank_taxa.difference(common_taxa)
    if (len(phlorest_taxa_only) != 0 or len(lexibank_taxa_only) != 0):
        print(phlorest_taxa_only)
        print(lexibank_taxa_only)
    
    
    phlorest_data = lang_data.loc[lang_data["verbose_name"] == row["phlorest_name"]].iloc[0]
    lexibank_data = lang_data.loc[lang_data["verbose_name"] == row["lexibank_name"]].iloc[0]
    t_phlorest = Tree(phlorest_data["newick_eval"])
    t_lexibank = Tree(lexibank_data["newick_eval"])
    #print(rf_distance_ete(t_phlorest, t_lexibank))
    
    

gray_and_atkinson2003.BIN.cc.phy
dyenindoeuropean.BIN.cc.phy
{'Tocharian_A', 'Tocharian_B', 'Hittite'}
{'91', '92', '89', '86', '94', '93', '95', '87', '90', '85', '88'}
koile_et_al2022.BIN.cc.phy
grollemundbantu.BIN.cc.phy
set()
{'d305nyangali', 'd308ebodo', 'd308bodo2', 'd304homa1919', 'd20bvamba1919'}
grollemund_et_al2015.BIN.cc.phy
grollemundbantu.BIN.cc.phy
kitchen_et_al2009.BIN.cc.phy
kitchensemitic.BIN.cc.phy
power_et_al2020.BIN.cc.phy
powerma.BIN.cc.phy
birchall_et_al2016.BIN.cc.phy
birchallchapacuran.BIN.cc.phy
gerardi_and_reichert2021.BIN.cc.phy
gerarditupi.BIN.cc.phy
greenhill_et_al_subm.BIN.cc.phy
utoaztecan.BIN.cc.phy
set()
{'SanJuanPuebloTewa', 'Kiowa', 'SantaDomingo', 'AztecPochutla', 'SantaAna', 'ProtoKeresan', 'Acoma'}
kolipakam_et_al2018.BIN.cc.phy
dravlex.BIN.cc.phy
lee2015.BIN.cc.phy
leekoreanic.BIN.cc.phy
lee_and_hasegawa2011.BIN.cc.phy
leejaponic.BIN.cc.phy
lee_and_hasegawa2013.BIN.cc.phy
leeainu.BIN.cc.phy
nagaraja_et_al2013.BIN.cc.phy
nagarajakhasian.BIN.cc.phy


A11_Londo,a11londo
A121_Mbonge,a121mbonge
A122_Bakundu,a122bakundu
A141_Lefo,a141lefo
A151_Nkongho,a151nkongho
A15A_Mbuu,a15ambuu
A15B_Mienge_NW_Manenguba,a15bmiengenwmanenguba
A15C_Akossi,a15cakossi
A15C_Elung_Central_Manenguba,a15celungcentralmanenguba
A15C_Mkaa,a15cmkaa
A15_Manenguba,a15manenguba
A22_Bakweri,a22bakweri
A24_Duala,a24duala
A27_Malimba,a27malimba
A31_Bubi,a31bubi
A32C_Batanga,a32cbatanga
A33a_Yasa,a33ayasa
A34_Benga,a34benga
A41_Barombi_Kang,a41barombikang
A42_Abo,a42abo
A43a_Basaa,a43abasaa
A44_Tunen,a44tunen
A45_Nyokon,a45nyokon
A462_Yambeta,a462yambeta
A46_Nomaande,a46nomaande
A51_Bafia_maja,a51bafiamaja
A53_Bafia_rikpa,a53bafiarikpa
A54_Tibea,a54tibea
A601_Tuki,a601tuki
A61_Ngoro_Asom,a61ngoroasom
A61_Ngoro_Bisoo,a61ngorobisoo
A61_Ngoro_lunda,a61ngorolunda
A621_Nubaca,a621nubaca
A622_Nugunu,a622nugunu
A62A_Nuasue,a62anuasue
A62A_Nukalonge,a62anukalonge
A62B_Mmala,a62bmmala
A62C_Libie,a62clibie
A63_Mangisa,a63mangisa
A71_Eton,a71eton
A72a_Ewondo,a72aewondo
A74a_Bulu

Aztec_Tetelcingo_30,
Aztec_Zacapoaxtla_31,
Cahuilla_12,
Chemehuevi_7,
Classical_Aztec_29,
Comanche_5,
Cora_27,
Cupeno_13,
Eudeve_21,
Gabrielino_15,
Guarijio_22,
Hopi_16,
Huichol_28,
Kawaiisu_6,
Kitanemuk_34,
Luiseno_14,
Mayo_25,
Mono_1,
Northern_Paiute_2,
Northern_Tepehuan_19,
Opata_20,
Pannamint_3,
Papago_Pima_17,
Pima_de_Onavas_18,
Pipil_32,
Serrano_11,
Shoshoni_Gosiute_Dialect_4,
Southeastern_Tepehuan_33,
Southern_Paiute_8,
Southern_Ute_9,
Tarahumara_23,
Tubar_24,
Tubatulabel_10,
Yaqui_26,
